In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [41]:
basedir = './국토교통부_주택_공시가격/'
filenames = [f for f in os.listdir(basedir)]
filenames

['2020년 공동주택 공시가격 정보.csv',
 'AL_11_D167_20201221_공동주택.csv',
 'AL_11_D169_20201027_단독주택.csv']

In [12]:
f = open(basedir + filenames[0], 'r')

In [13]:
f.readline()

'"기준연도","기준월","법정동코드","도로명주소","시도","시군구","읍면","동리","특수지코드","본번","부번","특수지명","단지명","동명","호명","전용면적","공시가격","단지코드","동코드","호코드"\n'

In [14]:
f.close()

In [15]:
official_house_price_df = pd.read_csv(basedir + filenames[0], encoding='ansi')
print(official_house_price_df.shape)
official_house_price_df.head()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (5,6,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(13829965, 20)


,기준연도,기준월,법정동코드,도로명주소,시도,시군구,읍면,동리,특수지코드,본번,부번,특수지명,단지명,동명,호명,전용면적,공시가격,단지코드,동코드,호코드
0,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,111,187.49,840000000,3,1,1
1,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,112,187.49,840000000,3,1,2
2,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,121,157.25,784000000,3,1,3
3,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,122,157.25,784000000,3,1,4
4,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,131,152.33,784000000,3,1,5


In [16]:
official_house_price_df['기준연도'].value_counts()

2020    13829965
Name: 기준연도, dtype: int64

In [18]:
f = open(basedir + filenames[1])

In [19]:
f.readline()

'고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건축물대장고유번호,기준년도,기준월,동코드,동명,토지대장면적,산정대지면적,건물전체연면적,건물산정연면적,주택가격,표준지여부,데이터기준일자\n'

In [20]:
f.readline()

'1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2005,01,10,10,112.4,112.4,33.5,33.5,70600000,N,2020-10-27\n'

In [21]:
f.close()

In [23]:
official_house_price_df2 = pd.read_csv(basedir + filenames[1], encoding='euc-kr')
print(official_house_price_df2.shape)
official_house_price_df2.head()

(6160646, 18)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건축물대장고유번호,기준년도,기준월,동코드,동명,토지대장면적,산정대지면적,건물전체연면적,건물산정연면적,주택가격,표준지여부,데이터기준일자
0,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2005,1,10,10,112.4,112.4,33.50,33.50,70600000,N,2020-10-27
1,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2006,1,10,10,112.4,112.4,33.50,33.50,83900000,N,2020-10-27
2,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2007,1,10,10,112.4,112.4,33.50,33.50,97400000,N,2020-10-27
3,1111010100100030057,1111010100,서울특별시 종로구 청운동,1,일반,3-57,1111010100100030057,2005,1,10,10,158.7,158.7,68.36,68.36,101000000,N,2020-10-27
4,1111010100100030057,1111010100,서울특별시 종로구 청운동,1,일반,3-57,1111010100100030057,2006,1,10,10,158.7,158.7,68.36,68.36,129000000,N,2020-10-27


In [35]:
chungun_df = official_house_price_df[(official_house_price_df['동리'] == '청운동')
                                    ]
print(chungun_df.shape)
chungun_df

(472, 20)


,기준연도,기준월,법정동코드,도로명주소,시도,시군구,읍면,동리,특수지코드,본번,부번,특수지명,단지명,동명,호명,전용면적,공시가격,단지코드,동코드,호코드
0,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,111,187.490,840000000,3,1,1
1,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,112,187.490,840000000,3,1,2
2,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,121,157.250,784000000,3,1,3
3,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,122,157.250,784000000,3,1,4
4,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,131,152.330,784000000,3,1,5
5,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,1,132,152.330,784000000,3,1,6
6,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,2,211,187.490,840000000,3,3,1
7,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,2,212,187.490,840000000,3,3,2
8,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,2,213,187.490,840000000,3,3,3
9,2020,1,1111010100,서울특별시 종로구 자하문로36길 16-14,서울특별시,종로구,NaN,청운동,0,1,0,NaN,청운벽산빌리지,2,214,187.490,840000000,3,3,4


In [29]:
official_house_price_df2.head(20)

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건축물대장고유번호,기준년도,기준월,동코드,동명,토지대장면적,산정대지면적,건물전체연면적,건물산정연면적,주택가격,표준지여부,데이터기준일자
0,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2005,1,10,10,112.4,112.4,33.50,33.50,70600000,N,2020-10-27
1,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2006,1,10,10,112.4,112.4,33.50,33.50,83900000,N,2020-10-27
2,1111010100100030047,1111010100,서울특별시 종로구 청운동,1,일반,3-47,1111010100100030047,2007,1,10,10,112.4,112.4,33.50,33.50,97400000,N,2020-10-27
3,1111010100100030057,1111010100,서울특별시 종로구 청운동,1,일반,3-57,1111010100100030057,2005,1,10,10,158.7,158.7,68.36,68.36,101000000,N,2020-10-27
4,1111010100100030057,1111010100,서울특별시 종로구 청운동,1,일반,3-57,1111010100100030057,2006,1,10,10,158.7,158.7,68.36,68.36,129000000,N,2020-10-27
5,1111010100100030057,1111010100,서울특별시 종로구 청운동,1,일반,3-57,1111010100100030057,2007,1,10,10,158.7,158.7,68.96,68.96,149000000,N,2020-10-27
6,1111010100100030059,1111010100,서울특별시 종로구 청운동,1,일반,3-59,1111010100100030059,2005,1,10,10,79.3,79.3,50.44,50.44,47100000,N,2020-10-27
7,1111010100100030059,1111010100,서울특별시 종로구 청운동,1,일반,3-59,1111010100100030059,2006,1,99999,99999,79.3,0.0,50.44,0.00,58900000,Y,2020-10-27
8,1111010100100030059,1111010100,서울특별시 종로구 청운동,1,일반,3-59,1111010100100030059,2007,1,99999,99999,79.3,0.0,50.44,0.00,61800000,Y,2020-10-27
9,1111010100100030060,1111010100,서울특별시 종로구 청운동,1,일반,3-60,1111010100100030060,2005,1,99999,99999,168.6,0.0,86.76,0.00,102000000,Y,2020-10-27


In [39]:
official_house_price_df2[(official_house_price_df2['법정동명'] == '서울특별시 종로구 청운동') 
                         & (official_house_price_df2['지번'] == '4-3')]

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건축물대장고유번호,기준년도,기준월,동코드,동명,토지대장면적,산정대지면적,건물전체연면적,건물산정연면적,주택가격,표준지여부,데이터기준일자
30,1111010100100040003,1111010100,서울특별시 종로구 청운동,1,일반,4-3,1111010100100040003,2005,1,10,10,899.0,743.92,506.71,419.3,1110000000,N,2020-10-27
31,1111010100100040003,1111010100,서울특별시 종로구 청운동,1,일반,4-3,1111010100100040003,2006,1,10,10,888.4,899.00,506.71,419.3,1390000000,N,2020-10-27


In [40]:
official_house_price_df2['특수지구분명'].value_counts()

일반    6156192
산        4454
Name: 특수지구분명, dtype: int64

In [42]:
gongdong_filenames = [f for f in os.listdir(basedir) if f.endswith('공동주택.csv')]
gongdong_filenames

['AL_11_D167_20201221_공동주택.csv']

In [46]:
f = open(basedir + gongdong_filenames[0])

In [47]:
f.readline()

'고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공동주택코드,공동주택구분코드,공동주택구분명,특수지명,공동주택명,동명,층명,호명,전용면적,공시가격,데이터기준일자\n'

In [48]:
f.readline()

'1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,01,3,3,빌라,,청운벽산빌리지,9,1,916,83.61,232000000,2020-03-09\n'

In [49]:
f.close()

In [50]:
gongdong_df = pd.read_csv(basedir + gongdong_filenames[0], encoding='euc-kr')
print(gongdong_df.shape)
gongdong_df.head()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (5,11,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(32685009, 19)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공동주택코드,공동주택구분코드,공동주택구분명,특수지명,공동주택명,동명,층명,호명,전용면적,공시가격,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,1,3,3,빌라,NaN,청운벽산빌리지,9,1,916,83.61,232000000,2020-03-09
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,1,3,3,빌라,NaN,청운벽산빌리지,9,2,921,83.61,240000000,2020-03-09
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,1,3,3,빌라,NaN,청운벽산빌리지,9,2,922,83.61,240000000,2020-03-09
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,1,3,3,빌라,NaN,청운벽산빌리지,9,2,923,83.61,240000000,2020-03-09
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2006,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1027,83.61,240000000,2020-03-09


In [52]:
basedir = './국토교통부_표준주택가격/'
filenames = [f for f in os.listdir(basedir)]
filenames

['2015 표준주택.csv',
 '2016 표준주택.csv',
 '2017 표준주택.csv',
 '2018 표준주택.csv',
 '2019 표준주택.CSV',
 '2020 표준주택.CSV']

In [71]:
f = open(basedir + filenames[0], 'r', encoding='utf-8')

In [72]:
f.readline()

'\ufeff시도,시군구,일련번호,소재지,도로명주소,지목,건물용도,대지면적,연면적,건축면적,용적률,건폐율,동수,층수(지상/지하),용도지역,용도지구,도로교통,형상지세,주건물구조,사용승인일,주위환경,주택공시가격\n'

In [73]:
f.readline()

'서울특별시,종로구,1, 청운동 4-18,자하문로36길 34  ,대,단독주택,1193.4,499.24,268.7,32.45,22.52,1,(2/1),제1종일반주거지역,자연경관지구,세로(가),정방형,연와조 ,1994,일반 주택지대,2960000000\n'

In [74]:
f.close()

In [75]:
standard_house_price_df = pd.read_csv(basedir + filenames[0])
print(standard_house_price_df.shape)
standard_house_price_df.head()

(189919, 22)


,시도,시군구,일련번호,소재지,도로명주소,지목,건물용도,대지면적,연면적,건축면적,용적률,건폐율,동수,층수(지상/지하),용도지역,용도지구,도로교통,형상지세,주건물구조,사용승인일,주위환경,주택공시가격
0,서울특별시,종로구,1,청운동 4-18,자하문로36길 34,대,단독주택,1193.4,499.24,268.70,32.45,22.52,1,(2/1),제1종일반주거지역,자연경관지구,세로(가),정방형,연와조,1994,일반 주택지대,2960000000
1,서울특별시,종로구,2,청운동 8-3,자하문로35길 5,대,단독주택,924.0,374.15,220.93,33.00,23.91,1,(2/1),제1종일반주거지역,자연경관지구,세로(가),사다리형,연와조 철근콘크리트조(RC조 RS조),1965,일반 주택지대,1960000000
2,서울특별시,종로구,3,청운동 12-5,자하문로36길 11-3,대,단독주택,118.7,77.81,57.02,56.50,48.04,1,(2/1),제1종일반주거지역,자연경관지구,소로한면,자루형,목조,1955,기존 주택지대,286000000
3,서울특별시,종로구,4,청운동 15-10,창의문로1길 20-5,대,단독주택,637.5,508.97,189.33,54.42,29.70,2,(2/1),제1종일반주거지역,자연경관지구,세로(가),사다리형,철근콘크리트조(RC조 RS조),1978,고급 주택지대,1840000000
4,서울특별시,종로구,5,청운동 38-1,자하문로 141,대,단독주택,357.9,181.08,98.46,46.30,27.51,1,(2/1),제1종일반주거지역,자연경관지구,광대한면,사다리형,연와조,1958,주택 및 상가혼용지대,1120000000


In [76]:
standard_house_price_df['건물용도'].value_counts()

단독주택        163849
다가구주택        19426
주상용건물         6479
다중주택           125
기타복합용건물         16
주산용건물           14
1종근린생활시설         5
2종근린생활시설         3
운동시설             1
주거용단독            1
Name: 건물용도, dtype: int64